### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [111]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### 1.  Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [114]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [115]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['take_revenge'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])
df_05.head(3)

,,syear,take_revenge,similar_problems,insult_back,rec1,rec2,rec3,rec4,rec5,rec6,rec7
pid,hid,,,,,,,,,,,
201,27,2005,[1] Trifft ueberhaupt nicht zu,[5] Skala 1-7,[1] Trifft ueberhaupt nicht zu,1,0,0,0,0,0,0
203,60313,2005,[2] Skala 1-7,[3] Skala 1-7,[2] Skala 1-7,0,1,0,0,0,0,0
602,60,2005,[5] Skala 1-7,[4] Skala 1-7,[3] Skala 1-7,0,0,0,0,1,0,0


In [116]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})

In [117]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001' ,  'xp5701' , 'xp0102']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job',  'xp5701' : 'commute_distance' , "xp0102" : "work_satisfaction"})

In [118]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil", "betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})

 ### 2. Define Functions and mappings for cleaning data

In [119]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for turnover intention: split up into binary with roughly equal value counts for simplicity: might change that later to categories
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention robustness check: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
# mapping for binary reciprocity measure
def binary_reciprocity(x):
    if x >= 3:
        return 1 # High Group
    else:
        return 0 # Low Group

## function for recoding values and dropping missing

def recode_categoricals(inputdf,rc_cardinal = 0):
    """
        merges data from different years, applies recoding to categoricals and constructs additional variables.

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode job satisfaction
    merged['work_satisfaction']= merged['work_satisfaction'].map(satisfaction_mapping)
    # recode turnover intention variable
    if rc_cardinal == 1:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping_cardinal)
    else:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen

def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df,rc_cardinal).astype('int')

    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan) 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
    # Convert 'gender' and 'sector' columns to categorical data type
    for col in ['gender']:
        df_mincer[col] = df_mincer[col].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

### Inlcude People who switched their jobs in 2006-2007 with 2006 controls

In [120]:
def include_jobchangers(data07,onlynewemployer=False,worksatisfaction=False):
    """
    Input: 
            df: finished 2007 dataframe
            onlynewemployer: optional argument: If True only includes those who switched jobs to a new employer
            worksatisfaction: if True drops observations where job satisfaction between 2006 and 2007 changed to check for our Assumption
    
    """
    df = data07
    # read in cleaned 2006 dataset:
    #  change path here : Maxie: '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    #                     Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'  
    observations_2006 = pd.read_csv('/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # drop s year columns from both dataframes
    df.drop(columns=df.filter(regex='^syear').columns, inplace=True)
    
    # optional worksatisfaction robustness check
    if worksatisfaction == True:
        satisfaction = pd.DataFrame(df['work_satisfaction']).join(pd.DataFrame(observations_2006['work_satisfaction']), on = ["pid", "hid"], lsuffix = "_07" , rsuffix ="_06")
        ID_keep_satis = satisfaction[satisfaction["work_satisfaction_07"] == satisfaction["work_satisfaction_06"]].index
        df.drop(df[~df.index.isin(ID_keep_satis)].index, inplace=True)
    
    # create list of IDs of people who switched to a new employer in the last year in 2007
    if onlynewemployer == True:
        IDs_tokeep = df[(df["new_job"] == 1) & (df["reason_new_job"] == '[3] Stelle bei neuen Arbeitgeber')].index
    # if True drops people whose work satisfaction changed between 2006 and 2007
    else:
        IDs_tokeep = df[(df["new_job"] == 1)].index
    
    # drop all who changed their job in 2007 dataframe and replace 2007 controls with 2006 controls
    df.drop(df[df["new_job"] == 1].index, inplace = True)
    observations_2006.drop(observations_2006[~observations_2006.index.isin(IDs_tokeep)].index, inplace=True)
    #concat both dataframes
    dfconcat = pd.concat([df,observations_2006])
    
    return dfconcat

### 3. Analysis
#### 1.Probit Regression

- Probit Estimation: TurnoverIntention_{2005} = Constant + Neg-Rec + Unfair + Rec X Unfair + Controls + Error

- As the measure for unfair treatment in the first regression i first used recog_effort -> decide for one later
    - "When I consider all my accomplishments and efforts, the recognition of I've received seems about right to me"

In [121]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS = df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
df_OLS["recXrecog_effort"] = df_OLS["recog_effort"] * df_OLS["avg_rec"]
# drop missing data in regression dataframe
df_OLS.dropna(inplace=True)

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


In [122]:
from statsmodels.discrete.discrete_model import Probit
# Define Outcome variable
Y = df_OLS["turnover_intention"]
# define X matrix
X = df_OLS.drop(columns=["turnover_intention"])
# add constant 
X = sm.add_constant(X)
# build model
model = Probit(Y, X.astype(float))
# estimate model
probit_model = model.fit()
# print summary table
probit_model.summary()

Optimization terminated successfully.
         Current function value: 0.578235
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:     turnover_intention   No. Observations:                 5962
Model:                         Probit   Df Residuals:                     5949
Method:                           MLE   Df Model:                           12
Date:                Sun, 08 Jan 2023   Pseudo R-squ.:                  0.1628
Time:                        18:53:03   Log-Likelihood:                -3447.4
converged:                       True   LL-Null:                       -4117.6
Covariance Type:            nonrobust   LLR p-value:                1.020e-279
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0500      0.193      0.259      0.795      -0.328       0.428
recog_effort             0.5002      0.089      5.611      0.000       0.325       0.675
working_hours           -0.0028      0.002     -1.178      0.239      -0.007       0.002
firmsize                -0.0340      0.006     -5.391      0.000      -0.046      -0.022
tenure                  -0.0311      0.002    -13.729      0.000      -0.036      -0.027
years_educ               0.0603      0.007      9.038      0.000       0.047       0.073
gender                   0.1464      0.042      3.509      0.000       0.065       0.228
commute_distance         0.0013      0.000      3.255      0.001       0.001       0.002
avg_rec                  0.0697      0.016      4.421      0.000       0.039       0.101
potential_experience    -0.0010      0.000     -2.903      0.004      -0.002      -0.000
age_squared              0.0177      0.021      0.839      0.401      -0.024       0.059
mincer_residuals     -5.588e-05   2.69e-05     -2.076      0.038      -0.000   -3.13e-06
recXrecog_effort        -0.0569      0.026     -2.217      0.027      -0.107      -0.007
========================================================================================
"""

#### 2. OLS
Now, instead of using a probit model, we use OLS.  linear probability model

In [123]:
from statsmodels.regression.linear_model import OLS
model = OLS(Y, X.astype(float))
lpm_model = model.fit()
lpm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     127.1
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          8.21e-284
Time:                        18:53:06   Log-Likelihood:                -3631.6
No. Observations:                5962   AIC:                             7289.
Df Residuals:                    5949   BIC:                             7376.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.5621      0.061      9.266      0.000       0.443       0.681
recog_effort             0.1657      0.029      5.662      0.000       0.108       0.223
working_hours           -0.0012      0.001     -1.487      0.137      -0.003       0.000
firmsize                -0.0113      0.002     -5.423      0.000      -0.015      -0.007
tenure                  -0.0103      0.001    -14.227      0.000      -0.012      -0.009
years_educ               0.0202      0.002      9.356      0.000       0.016       0.024
gender                   0.0505      0.014      3.680      0.000       0.024       0.077
commute_distance         0.0004      0.000      3.288      0.001       0.000       0.001
avg_rec                  0.0234      0.005      4.486      0.000       0.013       0.034
potential_experience    -0.0003      0.000     -2.583      0.010      -0.000   -6.79e-05
age_squared              0.0018      0.006      0.279      0.780      -0.011       0.014
mincer_residuals     -1.581e-05   8.78e-06     -1.800      0.072    -3.3e-05     1.4e-06
recXrecog_effort        -0.0188      0.008     -2.228      0.026      -0.035      -0.002
==============================================================================
Omnibus:                     5684.832   Durbin-Watson:                   1.757
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              391.756
Skew:                          -0.166   Prob(JB):                     8.54e-86
Kurtosis:                       1.789   Cond. No.                     9.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Robustness Checks


#### 1. Recode turnover variable into cardinal variable: Optional argument in recode_categoricals

In [124]:
# optional argument ==1
dfrc1 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal=1)))
# specify columns which we need for regression by dropping everything else
dfrc_cardinal = dfrc1.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
dfrc_cardinal["recXrecog_effort"] = dfrc_cardinal["recog_effort"] * dfrc_cardinal["avg_rec"]
# drop missing data in regression dataframe
dfrc_cardinal = dfrc_cardinal.dropna()

## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_cardinal["turnover_intention"]
X = dfrc_cardinal.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()

lpm_model.summary()

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     67.27
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          1.98e-154
Time:                        18:53:12   Log-Likelihood:                -27940.
No. Observations:                5962   AIC:                         5.591e+04
Df Residuals:                    5949   BIC:                         5.599e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   33.4064      3.578      9.337      0.000      26.392      40.421
recog_effort             7.8900      1.727      4.570      0.000       4.505      11.275
working_hours           -0.0752      0.046     -1.622      0.105      -0.166       0.016
firmsize                -0.2829      0.123     -2.309      0.021      -0.523      -0.043
tenure                  -0.3575      0.043     -8.335      0.000      -0.442      -0.273
years_educ               0.7148      0.127      5.612      0.000       0.465       0.965
gender                   2.4616      0.809      3.044      0.002       0.876       4.047
commute_distance         0.0142      0.007      2.097      0.036       0.001       0.028
avg_rec                  0.7448      0.307      2.426      0.015       0.143       1.347
potential_experience     0.0161      0.006      2.508      0.012       0.004       0.029
age_squared             -1.6599      0.382     -4.348      0.000      -2.408      -0.912
mincer_residuals        -0.0007      0.001     -1.255      0.209      -0.002       0.000
recXrecog_effort        -0.4278      0.497     -0.860      0.390      -1.403       0.547
==============================================================================
Omnibus:                     1184.351   Durbin-Watson:                   1.739
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2053.669
Skew:                           1.300   Prob(JB):                         0.00
Kurtosis:                       4.228   Cond. No.                     9.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Does not change much. Coefficient on the interaction term becomes insignificant.

#### 2. Different Reciprocity Specifications

    1. Split into binary: High/ Low reciprocal
    2. create dummies for each category and estimate regression for those


In [125]:
# 1. optional argument:  rc_binary ==1 . Split up from when average bigger than 3
dfrc2 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_rec_binary=1)))
# specify columns which we need for regression by dropping everything else
dfrc_binary = dfrc2.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age', 'avg_rec', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
# add interaction term
dfrc_binary["binaryrecXrecog_effort"] = dfrc_binary["recog_effort"] * dfrc_binary["binary_rec"]
# drop missing data in regression dataframe
dfrc_binary = dfrc_binary.dropna()
## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_binary["turnover_intention"]
X = dfrc_binary.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()
lpm_model.summary()

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     99.28
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          4.18e-223
Time:                        18:53:16   Log-Likelihood:                -3383.2
No. Observations:                5407   AIC:                             6792.
Df Residuals:                    5394   BIC:                             6878.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0.5105      0.064      7.924      0.000       0.384       0.637
recog_effort               0.1516      0.019      7.909      0.000       0.114       0.189
working_hours             -0.0009      0.001     -1.104      0.270      -0.003       0.001
firmsize                  -0.0119      0.002     -5.359      0.000      -0.016      -0.008
tenure                    -0.0086      0.001    -11.090      0.000      -0.010      -0.007
years_educ                 0.0198      0.002      8.612      0.000       0.015       0.024
gender                     0.0547      0.015      3.745      0.000       0.026       0.083
commute_distance           0.0004      0.000      2.922      0.003       0.000       0.001
binary_rec                 0.0700      0.016      4.456      0.000       0.039       0.101
potential_experience      -0.0004      0.000     -3.095      0.002      -0.001      -0.000
age_squared                0.0067      0.007      0.937      0.349      -0.007       0.021
mincer_residuals       -1.347e-05   9.36e-06     -1.440      0.150   -3.18e-05    4.88e-06
binaryrecXrecog_effort    -0.0686      0.026     -2.673      0.008      -0.119      -0.018
==============================================================================
Omnibus:                   114950.168   Durbin-Watson:                   1.834
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              414.217
Skew:                          -0.043   Prob(JB):                     1.13e-90
Kurtosis:                       1.647   Cond. No.                     1.01e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

##### also does not change a lot

### 2. Creating dummies and include 7 interaction terms : Decide for one of the 3 questions for simplicity

In [126]:
# 1. optional argument:  rc_binary ==1
dfrc2 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_rec_binary=1)))
# specify columns which we need for regression by dropping everything else
dfrc_dummy = dfrc2.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay','year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age', 'avg_rec'])
# add interaction terms
for col in ['rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7']:
    dfrc_dummy = dfrc_dummy.assign(**{col + '_X_recog_effort': dfrc_dummy[col] * dfrc_dummy['recog_effort']})
# drop missing data in regression dataframe
dfrc_dummy = dfrc_dummy.dropna()
## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_dummy["turnover_intention"]
X = dfrc_dummy.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()
lpm_model.summary()

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     57.93
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          2.56e-264
Time:                        18:53:21   Log-Likelihood:                -3230.7
No. Observations:                5372   AIC:                             6515.
Df Residuals:                    5345   BIC:                             6693.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0184      0.191      0.097      0.923      -0.355       0.392
rec1                     0.2005      0.182      1.100      0.271      -0.157       0.558
rec2                     0.3284      0.182      1.804      0.071      -0.029       0.685
rec3                     0.3817      0.182      2.097      0.036       0.025       0.739
rec4                     0.3259      0.182      1.789      0.074      -0.031       0.683
rec5                     0.3702      0.183      2.027      0.043       0.012       0.728
rec6                     0.3824      0.185      2.069      0.039       0.020       0.745
rec7                     0.2766      0.185      1.499      0.134      -0.085       0.638
recog_effort             0.5548      0.362      1.534      0.125      -0.154       1.264
working_hours           -0.0014      0.001     -1.610      0.107      -0.003       0.000
firmsize                -0.0116      0.002     -5.289      0.000      -0.016      -0.007
tenure                  -0.0086      0.001    -11.227      0.000      -0.010      -0.007
years_educ               0.0199      0.002      8.724      0.000       0.015       0.024
gender                   0.0544      0.014      3.781      0.000       0.026       0.083
commute_distance         0.0003      0.000      2.683      0.007    9.35e-05       0.001
work_satisfaction        0.1945      0.015     13.093      0.000       0.165       0.224
binary_rec              -0.0048      0.019     -0.252      0.801      -0.042       0.032
potential_experience    -0.0004      0.000     -3.221      0.001      -0.001      -0.000
age_squared              0.0071      0.007      1.002      0.316      -0.007       0.021
mincer_residuals      -5.67e-06   9.21e-06     -0.616      0.538   -2.37e-05    1.24e-05
rec1_X_recog_effort     -0.4467      0.363     -1.231      0.219      -1.158       0.265
rec2_X_recog_effort     -0.4592      0.363     -1.266      0.206      -1.170       0.252
rec3_X_recog_effort     -0.4546      0.363     -1.253      0.210      -1.166       0.257
rec4_X_recog_effort     -0.4839      0.363     -1.334      0.182      -1.195       0.227
rec5_X_recog_effort     -0.5391      0.364     -1.483      0.138      -1.252       0.174
rec6_X_recog_effort     -0.5039      0.365     -1.379      0.168      -1.220       0.213
rec7_X_recog_effort     -0.5133      0.366     -1.404      0.160      -1.230       0.203
==============================================================================
Omnibus:                     6951.807   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              345.017
Skew:                          -0.022   Prob(JB):                     1.20e-75
Kurtosis:                       1.759   Cond. No.                     1.67e+05
==================

##### Robustness Check : Drop people whose job satisfaction level changed

In [127]:
dfworksatisfation = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_rec_binary=1)),worksatisfaction = True)
# specify columns which we need for regression by dropping everything else
df_rc_ws = dfworksatisfation.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_rc_ws["recXrecog_effort"] = df_rc_ws["recog_effort"] * df_rc_ws["avg_rec"]
# drop missing data in regression dataframe
df_rc_ws.dropna(inplace=True)

from statsmodels.regression.linear_model import OLS

Y = df_rc_ws["turnover_intention"]
X = df_rc_ws.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()
lpm_model.summary()

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     67.79
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          2.81e-163
Time:                        18:53:25   Log-Likelihood:                -2744.1
No. Observations:                4346   AIC:                             5516.
Df Residuals:                    4332   BIC:                             5605.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.4405      0.076      5.827      0.000       0.292       0.589
recog_effort             0.2040      0.036      5.691      0.000       0.134       0.274
working_hours           -0.0008      0.001     -0.863      0.388      -0.003       0.001
firmsize                -0.0113      0.003     -4.490      0.000      -0.016      -0.006
tenure                  -0.0077      0.001     -8.781      0.000      -0.009      -0.006
years_educ               0.0211      0.003      8.212      0.000       0.016       0.026
gender                   0.0495      0.016      3.002      0.003       0.017       0.082
commute_distance         0.0004      0.000      2.379      0.017    6.46e-05       0.001
avg_rec                  0.0266      0.009      2.954      0.003       0.009       0.044
binary_rec              -0.0011      0.023     -0.048      0.962      -0.046       0.044
potential_experience    -0.0003      0.000     -2.173      0.030      -0.001   -2.85e-05
age_squared              0.0024      0.008      0.300      0.764      -0.013       0.018
mincer_residuals     -8.208e-06   1.06e-05     -0.775      0.438    -2.9e-05    1.26e-05
recXrecog_effort        -0.0253      0.010     -2.428      0.015      -0.046      -0.005
==============================================================================
Omnibus:                    45563.699   Durbin-Watson:                   1.845
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              352.570
Skew:                           0.029   Prob(JB):                     2.76e-77
Kurtosis:                       1.606   Cond. No.                     1.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Robustness Check 

Avg reciprocity measure over years



In [128]:
df_OLS_avg = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS_avg = df_OLS_avg.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'age', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction','avg_rec'])
# Load avg reciprocity measures over the years

avg_reciprocity = pd.read_csv('/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/rec_avgyears.csv')
avg_reciprocity.reset_index(inplace=True)
avg_reciprocity.set_index(["pid","hid"], inplace=True)
avg_reciprocity.drop("index",axis=1 ,inplace =True)

df_avg_years = pd.merge(df_OLS_avg,avg_reciprocity, on=["pid","hid"])
# add interaction term
df_avg_years["recXrecog_effort"] = df_avg_years["recog_effort"] * df_avg_years["avg_rec"]
# drop missing data in regression dataframe
df_avg_years.dropna(inplace=True)




from statsmodels.regression.linear_model import OLS

Y = df_avg_years["turnover_intention"]
X = df_avg_years.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()
lpm_model.summary()



/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:193: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_85345/2434218017.py:251: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     127.2
Date:                Sun, 08 Jan 2023   Prob (F-statistic):          6.21e-284
Time:                        18:53:28   Log-Likelihood:                -3631.3
No. Observations:                5962   AIC:                             7289.
Df Residuals:                    5949   BIC:                             7376.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.5509      0.061      8.968      0.000       0.430       0.671
recog_effort             0.1604      0.032      5.021      0.000       0.098       0.223
working_hours           -0.0012      0.001     -1.505      0.132      -0.003       0.000
firmsize                -0.0112      0.002     -5.382      0.000      -0.015      -0.007
tenure                  -0.0104      0.001    -14.247      0.000      -0.012      -0.009
years_educ               0.0206      0.002      9.501      0.000       0.016       0.025
gender                   0.0491      0.014      3.571      0.000       0.022       0.076
commute_distance         0.0004      0.000      3.301      0.001       0.000       0.001
potential_experience    -0.0003      0.000     -2.551      0.011      -0.000   -6.44e-05
age_squared              0.0016      0.006      0.253      0.801      -0.011       0.014
mincer_residuals     -1.565e-05   8.78e-06     -1.782      0.075   -3.29e-05    1.57e-06
avg_rec                  0.0260      0.006      4.471      0.000       0.015       0.037
recXrecog_effort        -0.0173      0.009     -1.850      0.064      -0.036       0.001
==============================================================================
Omnibus:                     5582.654   Durbin-Watson:                   1.756
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              390.089
Skew:                          -0.165   Prob(JB):                     1.96e-85
Kurtosis:                       1.791   Cond. No.                     1.00e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  1e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Also does not change much